In [1]:
%pip install kaggle
%pip install numpy
%pip install pandas
%pip install scikit-learn
%pip install polars
%pip install pyarrow
%pip install seaborn
%pip install matplotlib
%pip install xgboost
%pip install psutil

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Create pipeline

In [1]:
import polars as pl

class Pipeline: 
    def __init__(
        self,
        data_path: str,
        null_threshold: float = 0.7,
        high_card_threshold: int = 200,
    ):
        self.data_path = data_path
        self.null_threshold = null_threshold
        self.high_card_threshold = high_card_threshold
        self.feature_columns = None
        
    def load_data(self, filename):
        df = pl.read_parquet(f"{self.data_path}/{filename}")
        return df
    
    def set_table_dtypes(self, df):
        cast_exprs = []

        for col in df.columns:

            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                cast_exprs.append(pl.col(col).cast(pl.Int64))

            elif col == "date_decision":
                cast_exprs.append(pl.col(col).cast(pl.Date))

            elif col.endswith(("P", "A")):
                cast_exprs.append(pl.col(col).cast(pl.Float32))  # use Float32 for memory

            elif col.endswith("M"):
                cast_exprs.append(pl.col(col).cast(pl.Utf8))

            elif col.endswith("D"):
                cast_exprs.append(pl.col(col).cast(pl.Date))

        return df.with_columns(cast_exprs)
    
    def handle_dates(self, df: pl.DataFrame) -> pl.DataFrame:
        
        if "date_decision" not in df.columns:
            return df
        
        date_cols = [c for c in df.columns if c.endswith("D")]

        df = df.with_columns([
            (
                pl.col(col) - pl.col("date_decision")
            )
            .dt.total_days()
            .cast(pl.Float32)
            .alias(col)
            for col in date_cols
        ])

        # Drop decision date & MONTH if exists
        drop_cols = [c for c in ["date_decision", "MONTH"] if c in df.columns]
        if drop_cols:
            df = df.drop(drop_cols)

        return df

    # Remove high-null columns and remove high-cardinality string columns
    def filter_columns(self, df: pl.DataFrame) -> pl.DataFrame:
        cols_to_drop = []

        protected = ["target", "case_id", "WEEK_NUM"]

        for col in df.columns:

            if col in protected:
                continue

            # High null ratio filtering
            null_ratio = (
                df.select(pl.col(col).null_count() / pl.count()).item()
            )

            if null_ratio > self.null_threshold:
                cols_to_drop.append(col)
                continue

            # String cardinality filtering
            if df.schema[col] == pl.Utf8:
                n_unique = df.select(pl.col(col).n_unique()).item()

                if n_unique == 1 or n_unique > self.high_card_threshold:
                    cols_to_drop.append(col)

        if cols_to_drop:
            df = df.drop(cols_to_drop)

        return df
    
    def preprocess(self, filename: str) -> pl.DataFrame:
        df = self.load_data(filename)
        df = self.set_table_dtypes(df)
        df = self.handle_dates(df)
        df = self.filter_columns(df)
        return df
    

    def fit(self, train_filename: str):

        df = self.preprocess(train_filename)

        self.y = df["target"].to_numpy()
        df = df.drop("target")

        self.feature_columns = df.columns

        return df
    
    def transform(self, test_filename: str):

        df = self.preprocess(test_filename)

        # Align to train feature order
        df = df.select(self.feature_columns)

        return df

# Create aggregator

In [2]:
class Aggregator:
    def num_expr(df, prefix):
        cols = [c for c in df.columns if c.endswith(("P", "A"))]

        return (
            [pl.col(c).max().alias(f"{prefix}_max_{c}") for c in cols] +
            [pl.col(c).mean().alias(f"{prefix}_mean_{c}") for c in cols] +
            [pl.col(c).last().alias(f"{prefix}_last_{c}") for c in cols]
        )
    
    def date_expr(df, prefix):
        cols = [c for c in df.columns if c.endswith("D")]

        return (
            [pl.col(c).max().alias(f"{prefix}_max_{c}") for c in cols] +
            [pl.col(c).last().alias(f"{prefix}_last_{c}") for c in cols]
        )
    
    def str_expr(df, prefix):
        cols = [c for c in df.columns if c.endswith("M")]

        return (
            [pl.col(c).last().alias(f"{prefix}_last_{c}") for c in cols]
        )
    
    def count_expr(df, prefix):
        cols = [c for c in df.columns if "num_group" in c]

        return (
            [pl.col(c).max().alias(f"{prefix}_max_{c}") for c in cols] +
            [pl.col(c).last().alias(f"{prefix}_last_{c}") for c in cols]
        )
    
    def other_expr(df, prefix):
        cols = [c for c in df.columns if c.endswith(("T", "L"))]

        return (
            [pl.col(c).max().alias(f"{prefix}_max_{c}") for c in cols] +
            [pl.col(c).last().alias(f"{prefix}_last_{c}") for c in cols]
        )
    
    def get_exprs(df, prefix):

        return (
            Aggregator.num_expr(df, prefix) +
            Aggregator.date_expr(df, prefix) +
            Aggregator.str_expr(df, prefix) +
            Aggregator.other_expr(df, prefix) +
            Aggregator.count_expr(df, prefix)
        )

# Load data

In [18]:
from pathlib import Path

DATA_DIR = Path("home_credit_data/parquet_files/train")

TABLES = {
    "base": DATA_DIR / "train_base.parquet",
    "static_0_0": DATA_DIR / "train_static_0_0.parquet",
    "static_0_1": DATA_DIR / "train_static_0_1.parquet",
    "static_cb_0": DATA_DIR / "train_static_cb_0.parquet",
    "person_1": DATA_DIR / "train_person_1.parquet",
    "person_2": DATA_DIR / "train_person_2.parquet",
    "applprev_1_0": DATA_DIR / "train_applprev_1_0.parquet",
    "applprev_1_1": DATA_DIR / "train_applprev_1_1.parquet",
    "applprev_2": DATA_DIR / "train_applprev_2.parquet",
    "credit_bureau_a_1_0": DATA_DIR / "train_credit_bureau_a_1_0.parquet",
    "credit_bureau_a_1_1": DATA_DIR / "train_credit_bureau_a_1_1.parquet",
    "credit_bureau_a_1_2": DATA_DIR / "train_credit_bureau_a_1_2.parquet",
    "credit_bureau_a_1_3": DATA_DIR / "train_credit_bureau_a_1_3.parquet",
    "credit_bureau_a_2_0": DATA_DIR / "train_credit_bureau_a_2_0.parquet",
    "credit_bureau_a_2_1": DATA_DIR / "train_credit_bureau_a_2_1.parquet",
    "credit_bureau_a_2_2": DATA_DIR / "train_credit_bureau_a_2_2.parquet",
    "credit_bureau_a_2_3": DATA_DIR / "train_credit_bureau_a_2_3.parquet",
    "credit_bureau_a_2_4": DATA_DIR / "train_credit_bureau_a_2_4.parquet",
    "credit_bureau_a_2_5": DATA_DIR / "train_credit_bureau_a_2_5.parquet",
    "credit_bureau_a_2_6": DATA_DIR / "train_credit_bureau_a_2_6.parquet",
    "credit_bureau_a_2_7": DATA_DIR / "train_credit_bureau_a_2_7.parquet",
    "credit_bureau_a_2_8": DATA_DIR / "train_credit_bureau_a_2_8.parquet",
    "credit_bureau_a_2_9": DATA_DIR / "train_credit_bureau_a_2_9.parquet",
    "credit_bureau_a_2_10": DATA_DIR / "train_credit_bureau_a_2_10.parquet",
    "credit_bureau_b_1": DATA_DIR / "train_credit_bureau_b_1.parquet",
    "credit_bureau_b_2": DATA_DIR / "train_credit_bureau_b_2.parquet",
    "debitcard_1": DATA_DIR / "train_debitcard_1.parquet",
    "deposit_1": DATA_DIR / "train_deposit_1.parquet",
    "other_1": DATA_DIR / "train_other_1.parquet",
    "tax_registry_a_1": DATA_DIR / "train_tax_registry_a_1.parquet",
    "tax_registry_b_1": DATA_DIR / "train_tax_registry_b_1.parquet",
    "tax_registry_c_1": DATA_DIR / "train_tax_registry_c_1.parquet",
}

TEST_DATA_DIR = Path("home_credit_data/parquet_files/test")
TEST_TABLES = {
        "base": TEST_DATA_DIR / "test_base.parquet",
        "static_0_0": TEST_DATA_DIR / "test_static_0_0.parquet",
        "static_0_1": TEST_DATA_DIR / "test_static_0_1.parquet",
        "static_cb_0": TEST_DATA_DIR / "test_static_cb_0.parquet",
        "person_1": TEST_DATA_DIR / "test_person_1.parquet",
        "person_2": TEST_DATA_DIR / "test_person_2.parquet",
        "applprev_1_0": TEST_DATA_DIR / "test_applprev_1_0.parquet",
        "applprev_1_1": TEST_DATA_DIR / "test_applprev_1_1.parquet",
        "applprev_2": TEST_DATA_DIR / "test_applprev_2.parquet",
        "credit_bureau_a_1_0": TEST_DATA_DIR / "test_credit_bureau_a_1_0.parquet",
        "credit_bureau_a_1_1": TEST_DATA_DIR / "test_credit_bureau_a_1_1.parquet",
        "credit_bureau_a_1_2": TEST_DATA_DIR / "test_credit_bureau_a_1_2.parquet",
        "credit_bureau_a_1_3": TEST_DATA_DIR / "test_credit_bureau_a_1_3.parquet",
        "credit_bureau_a_2_0": TEST_DATA_DIR / "test_credit_bureau_a_2_0.parquet",
        "credit_bureau_a_2_1": TEST_DATA_DIR / "test_credit_bureau_a_2_1.parquet",
        "credit_bureau_a_2_2": TEST_DATA_DIR / "test_credit_bureau_a_2_2.parquet",
        "credit_bureau_a_2_3": TEST_DATA_DIR / "test_credit_bureau_a_2_3.parquet",
        "credit_bureau_a_2_4": TEST_DATA_DIR / "test_credit_bureau_a_2_4.parquet",
        "credit_bureau_a_2_5": TEST_DATA_DIR / "test_credit_bureau_a_2_5.parquet",
        "credit_bureau_a_2_6": TEST_DATA_DIR / "test_credit_bureau_a_2_6.parquet",
        "credit_bureau_a_2_7": TEST_DATA_DIR / "test_credit_bureau_a_2_7.parquet",
        "credit_bureau_a_2_8": TEST_DATA_DIR / "test_credit_bureau_a_2_8.parquet",
        "credit_bureau_a_2_9": TEST_DATA_DIR / "test_credit_bureau_a_2_9.parquet",
        "credit_bureau_a_2_10": TEST_DATA_DIR / "test_credit_bureau_a_2_10.parquet",
        "credit_bureau_b_1": TEST_DATA_DIR / "test_credit_bureau_b_1.parquet",
        "credit_bureau_b_2": TEST_DATA_DIR / "test_credit_bureau_b_2.parquet",
        "debitcard_1": TEST_DATA_DIR / "test_debitcard_1.parquet",
        "deposit_1": TEST_DATA_DIR / "test_deposit_1.parquet",       
        "other_1": TEST_DATA_DIR / "test_other_1.parquet",
        "tax_registry_a_1": TEST_DATA_DIR / "test_tax_registry_a_1.parquet",
        "tax_registry_b_1": TEST_DATA_DIR / "test_tax_registry_b_1.parquet",
        "tax_registry_c_1": TEST_DATA_DIR / "test_tax_registry_c_1.parquet",
}

In [4]:
def reduce_memory(df: pl.DataFrame, verbose: bool = True) -> pl.DataFrame:

    start_mem = df.estimated_size("mb")

    cast_exprs = []

    for col, dtype in df.schema.items():

        # ---- Integers
        if dtype == pl.Int64:
            col_min = df[col].min()
            col_max = df[col].max()

            if col_min >= -128 and col_max <= 127:
                cast_exprs.append(pl.col(col).cast(pl.Int8))
            elif col_min >= -32768 and col_max <= 32767:
                cast_exprs.append(pl.col(col).cast(pl.Int16))
            elif col_min >= -2147483648 and col_max <= 2147483647:
                cast_exprs.append(pl.col(col).cast(pl.Int32))

        # ---- Floats
        elif dtype == pl.Float64:
            cast_exprs.append(pl.col(col).cast(pl.Float32))

        # ---- Strings (optional: convert low-cardinality to categorical)
        elif dtype == pl.Utf8:
            n_unique = df[col].n_unique()
            if n_unique < 0.5 * len(df):
                cast_exprs.append(pl.col(col).cast(pl.Categorical))

    if cast_exprs:
        df = df.with_columns(cast_exprs)

    end_mem = df.estimated_size("mb")

    if verbose:
        print(f"Memory before: {start_mem:.2f} MB")
        print(f"Memory after : {end_mem:.2f} MB")
        print(f"Reduced by    : {(start_mem - end_mem):.2f} MB "
              f"({100*(start_mem - end_mem)/start_mem:.1f}%)")

    return df

In [27]:
# Load and preprocess data
import gc


train_base = pl.read_parquet(TABLES["base"])
train_base = Pipeline.set_table_dtypes(None, train_base)
train_base = Pipeline.handle_dates(None, train_base)
train_base = reduce_memory(train_base)

train_merged = train_base
# merge all tables with depth 1
tables = ["person_1", "static_0_0", "static_0_1", "static_cb_0", "other_1"]
for table in tables:
    table_data = pl.read_parquet(TABLES[table])
    table_data = Pipeline.set_table_dtypes(None, table_data) 
    table_data = Pipeline.handle_dates(None, table_data)
    table_data = (table_data
        .group_by("case_id")
        .agg(Aggregator.get_exprs(table_data, prefix=table))
    )
    table_data = reduce_memory(table_data)
    train_merged = train_merged.join(table_data, on="case_id", how="left")
    del table_data
    gc.collect()

train_merged = reduce_memory(train_merged)
train_merged.shape

Memory before: 34.94 MB
Memory after : 8.74 MB
Reduced by    : 26.21 MB (75.0%)
Memory before: 393.91 MB
Memory after : 327.24 MB
Reduced by    : 66.67 MB (16.9%)
Memory before: 2025.64 MB
Memory after : 1444.19 MB
Reduced by    : 581.45 MB (28.7%)
Memory before: 1054.20 MB
Memory after : 752.09 MB
Reduced by    : 302.11 MB (28.7%)
Memory before: 1004.32 MB
Memory after : 612.81 MB
Reduced by    : 391.50 MB (39.0%)
Memory before: 4.09 MB
Memory after : 3.22 MB
Reduced by    : 0.88 MB (21.4%)
Memory before: 5459.21 MB
Memory after : 5459.21 MB
Reduced by    : 0.00 MB (0.0%)


(1526659, 946)

In [31]:
print(train_base["case_id"].n_unique())
print(train_merged.estimated_size("mb"))

1526659
5459.212637901306


In [32]:
n_rows = train_merged.shape[0]

cols_keep = [
    c for c in train_merged.columns
    if train_merged[c].null_count() / n_rows < 0.8
]

train_merged = train_merged.select(cols_keep)

print(train_merged.shape)

(1526659, 667)


In [33]:
numeric_cols = [
    c for c in train_merged.columns
    if train_merged[c].dtype in (pl.Float32, pl.Float64, pl.Int32, pl.Int64)
]

train_merged = train_merged.select(numeric_cols + ["target"])


In [34]:
train_merged.shape

(1526659, 537)

In [35]:
import numpy as np

y = train_merged["target"].to_numpy()
X = train_merged.drop("target").to_numpy().astype(np.float32)
feature_columns = train_merged.drop("target").columns

del train_merged
gc.collect()

print(X.shape)

(1526659, 536)


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=800,           
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",         
    eval_metric="auc",
    random_state=42,
    n_jobs=-1
)

# X_train, X_valid, y_train, y_valid = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )

# model.fit(X_train, y_train)

# preds = model.predict_proba(X_valid)[:, 1]
# auc = roc_auc_score(y_valid, preds)

# print("Validation AUC:", auc)

# Validation AUC: 0.8052800470575214

In [ ]:
# import joblib
# # Train on full datasets
# model.fit(X, y)

# joblib.dump(model, "xgb_full_model.pkl")
# joblib.dump(feature_columns, "feature_columns.pkl")

# gc.collect()

30

In [ ]:
test_merged = pl.read_parquet(TEST_TABLES["base"])

tables = ["person_1", "static_0_0", "static_0_1", "static_cb_0", "other_1"]
for table in tables:
    table_data = pl.read_parquet(TEST_TABLES[table])
    table_data = Pipeline.set_table_dtypes(None, table_data) 
    table_data = Pipeline.handle_dates(None, table_data)
    table_data = (table_data
        .group_by("case_id")
        .agg(Aggregator.get_exprs(table_data, prefix=table))
    )
    table_data = reduce_memory(table_data)
    test_merged = test_merged.join(table_data, on="case_id", how="left")
    del table_data
    gc.collect()

test_merged.shape


Memory before: 0.00 MB
Memory after : 0.00 MB
Reduced by    : 0.00 MB (17.7%)
Memory before: 0.02 MB
Memory after : 0.01 MB
Reduced by    : 0.01 MB (27.9%)
Memory before: 0.02 MB
Memory after : 0.01 MB
Reduced by    : 0.01 MB (27.7%)
Memory before: 0.01 MB
Memory after : 0.00 MB
Reduced by    : 0.00 MB (40.3%)
Memory before: 0.00 MB
Memory after : 0.00 MB
Reduced by    : 0.00 MB (21.4%)
['target']


In [37]:
import joblib
import numpy as np
import pandas as pd

model = joblib.load("xgb_full_model.pkl")
feature_columns = joblib.load("feature_columns.pkl")

common_features = [c for c in feature_columns if c in test_merged.columns]


X_test = (
    test_merged
    .select(common_features)        # ensure correct order
    .to_numpy()
    .astype(np.float32)
)

print("Test shape:", X_test.shape)
print("Expected features:", len(feature_columns))

Test shape: (10, 536)
Expected features: 536


In [38]:
import numpy as np

test_preds = model.predict_proba(X_test)[:, 1]

test_preds = np.asarray(test_preds, dtype=float)
test_preds = np.nan_to_num(test_preds, nan=0.5, posinf=1.0, neginf=0.0)
test_preds = np.clip(test_preds, 0.0, 1.0)

print("Prediction shape:", test_preds.shape)

Prediction shape: (10,)


In [ ]:
submission = pd.DataFrame({
    "case_id": test_merged["case_id"].to_numpy(),
    "score": test_preds
})

submission.to_csv("submission.csv", index=False)

print("Submission saved successfully!")

del model, feature_columns, X_test, test_preds, submission
gc.collect()

Submission saved successfully!
